<a href="https://colab.research.google.com/github/Kaveesha20/drop-cable-length-detector/blob/main/llama3_2_vision_11b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl https://ollama.ai/install.sh | sh
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  57558      0 --:--:-- --:--:-- --:--:-- 57743
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jamm

In [3]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia'

In [4]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [5]:
!ollama pull llama3.2-vision:11b

In [6]:
!pip install ollama

In [7]:
import ollama
import re
from google.colab import files
from IPython.display import display
from PIL import Image
import io

In [14]:
print("📂 Please upload exactly two image files (e.g., handwritten fiber lengths)...")
uploaded = files.upload()

image_paths = list(uploaded.keys())
print(f"✅ Uploaded files: {image_paths}")


📂 Please upload exactly two image files (e.g., handwritten fiber lengths)...


Saving 1751103594480.jpg to 1751103594480.jpg
Saving 1751107327044.jpg to 1751107327044.jpg
✅ Uploaded files: ['1751103594480.jpg', '1751107327044.jpg']


In [15]:
def extract_number_from_image_bytes(image_bytes, image_name='uploaded_image'):
    response = ollama.chat(
        model='llama3.2-vision:11b',
        messages=[{
            'role': 'user',
            'content': 'Extract the handwritten number in meters from this image.',
            'images': [image_bytes]
        }]
    )

    content = response['message']['content']
    print(f"\n📝 Raw model output for {image_name}:\n{content}")

    match = re.search(r'(\d+(?:\.\d+)?)(?:\s*m| meters)?', content.lower())
    if match:
        return float(match.group(1))
    else:
        print(f"❌ No number found in {image_name}")
        return None


In [16]:
if len(image_paths) != 2:
    print("⚠️ Please upload exactly two image files.")
else:
    num1 = extract_number_from_image_bytes(uploaded[image_paths[0]], image_paths[0])
    num2 = extract_number_from_image_bytes(uploaded[image_paths[1]], image_paths[1])

    if num1 is not None and num2 is not None:
        diff = abs(num1 - num2)
        print(f"\n📏 Fiber length difference: {diff} meters")
    else:
        print("\n⚠️ Could not calculate difference due to missing number(s).")



📝 Raw model output for 1751103594480.jpg:
The handwritten number in meters is 1854.

📝 Raw model output for 1751107327044.jpg:
The handwritten number in meters from the image is 54671.

📏 Fiber length difference: 52817.0 meters
